### Luigi Module

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 

pars = load_parameters()

input_file = pars.get('input')
output_file = pars.get('output')

# Dealing with multiple outputs

dataset = pars.get('dataset')

debug = pars.get('debug')

In [ ]:
import pandas as pd

In [ ]:
# input_file = '../../data/stack-overflow/pandas-solutioncode-h1'
# output_file = '../../data/results/results-baseline.txt'
# dataset = '../../data/stack-overflow/Dataset - Pandas.csv'

dataset = pd.read_csv(dataset, encoding='ISO-8859-1', error_bad_lines=False)

result_df = pd.read_pickle(input_file)


#### buildTaggedDatasetDSForEvaluation:

This method is used to build marker for tagged dataset. 
A post can have mutiple solutions and a solution can have multiple lines.

\[Id, List of alternative solutions\]
List of alternative solutions --> List of \[line, Marked_Flag\] .. Defualt mark is False

AnswerId: 41386927	

Sol 1= 	df = pd.read_csv('filename.txt', sep=";", names=['Region Name'])

Sol	2=	"df.insert(0, 'State', df['Region Name'].str.extract('(.*)\[edit\]', expand=False).ffill())

        df['Region Name'] = df['Region Name'].str.replace(r' \(.+$', '')"

A dictionary element
{41386927: [

            List of alternative solutions
            [   
                List of lines of Sol 1 
               [
               A pair which has line and Marker Flag sol 1 line 1
                   
                   [df = pd.read_csv('filename.txt', sep=";", names=['Region Name']), False]           
                ]                
                 List of lines of Sol 2
               [
               A pair which has line and Marker Flag sol 2 line 1
                   
                   [df.insert(0, 'State', df['Region Name'].str.extract('(.*)\[edit\]', expand=False).ffill()), False] ,
                   
               A pair which has line and Marker Flag sol 2 line 2
                   
                   [df['Region Name'] = df['Region Name'].str.replace(r' \(.+$', ''), False]
                ]
              ]
             ]
}
     


#### buildResultSetForEvaluation:

This method is used to build marker for result which can have multiple lines.

\[Id, List of \[line, Marked_Flag\]\] .. Defualt mark is False


#### markTheLines

This method iterate over the lines of result and if that line is present in the tagged solution dataset for corresponding answer, then mark the line in result and tagged dataset dictionary as True

In [ ]:
def buildTaggedDatasetDSForEvaluation(tagged_dataset_df):
    taggedDSEvalDict = dict()
    
    for index, row in tagged_dataset_df.iterrows():
        
        try:
        
            Id = row['AnswerId']
            if Id != 0:

                if Id in taggedDSEvalDict:
                    solutionList = taggedDSEvalDict[Id]
                else:
                    solutionList = list()                    
                lineList = list()
                lines = row['Solution'].split('\n')
                for line in lines:
                    if line.strip() != '':
                        pair = [line, False]
                        lineList.append(pair)
                solutionList.append(lineList)
                taggedDSEvalDict[Id]= solutionList

        except Exception as e:
            print(e)
    return taggedDSEvalDict


def buildResultSetForEvaluation(result_df):
    resultSetDict = dict()
    for index, row in result_df.iterrows():
        try:
            ansId = row['Id']
            solutionLinesTuple = list()                    
            predictedLines = row['Solution']
            for line in predictedLines:
                if line.strip() != '':
                    pair = [line, False]
                    solutionLinesTuple.append(pair)
            resultSetDict[ansId]= solutionLinesTuple

        except Exception as e:
            print(ansId, e)
    return resultSetDict

def markTheLines(taggedDSEvalDict, resultSetDict):
    for Id, pairs in resultSetDict.items():
        try:
            solutionList = taggedDSEvalDict[Id] # Error
            for respair in pairs:
                for solution in solutionList:
                    for tagpair in solution:
                        if tagpair[0].strip() == respair[0].strip():
                            tagpair[1] = True
                            respair[1] = True
        except KeyError:
            pass # Ignore it for now
    return taggedDSEvalDict, resultSetDict

def outputResultAsCSV(taggedDSEvalDict, resultSetDict):
    COLUMN_NAMES = ['Id', 'Matched_Prediction', 'Unmatched_Prediction', 'Matched_Actual', 'Unmatched_Actual']
    row_id = 0
    output_df = pd.DataFrame(columns=COLUMN_NAMES)
    for Id, pairs in resultSetDict.items():
        try:
            mps = ''
            umps = ''
            for respair in pairs:
                if respair[1]:
                    mps = mps + respair[0].strip() +  '\n'
                else:
                    umps = umps + respair[0].strip() +  '\n'
                    
            solutionList = taggedDSEvalDict[Id] # Error
            for solution in solutionList:
                mas = ''
                umas = ''
                for tagpair in solution:
                    if tagpair[1]:
                        mas = mas + tagpair[0].strip() +  '\n'
                    else:
                        umas = umas + respair[0].strip() +  '\n'
                output_df.loc[row_id] = [Id, mps, umps, mas, umas]
                row_id = row_id + 1
        except KeyError:
            pass # Ignore it for now
    return output_df

taggedDSEvalDict =  buildTaggedDatasetDSForEvaluation(dataset)
resultSetDict = buildResultSetForEvaluation(result_df)
taggedDSEvalDict, resultSetDict = markTheLines(taggedDSEvalDict, resultSetDict)
output_df = outputResultAsCSV(taggedDSEvalDict, resultSetDict)

##### Evaluation method

1. Check for all the lines in Predicted solution. If any of the line is marked False, means we are identifying few nonsolution lines as solution. Marking such cases as FP.

    For this defined a boolean variable MarkedPredicted (MP) which takes the AND of all lines of predicted solution. If all True then proceed to check the tagged dataset for actual solution, otherwise increase FP count for that Id.

2. If all the predicted lines are part of solution that means all marked as true, MP will be true.

    Check alternative solutions:

    For each solution for a particular answer, again defined MAS, MarkedActualSubsolution which is again 'AND' of all the lines of a specific solution. This means that all the lines in any alternative solution must be predicted by our system.

    Then we take 'OR'  of all the MAS as MarkedActual(MA) which says that if any of the actual solution is identified fully, we got a correct case and increase TP else again increase FP

In [ ]:
def evaluate(result_df):
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    
    for Id, pairs in resultSetDict.items():
        try:
            solutionList = taggedDSEvalDict[Id] # Error
            for respair in pairs:
                for solution in solutionList:
                    for tagpair in solution:
                        # TRUE POSITIVE - Pair predicted was found in the solution
                        if respair[1]:
                            TP = TP + 1

                        # FALSE POSITIVE  - Pair predicted was NOT found in the solution
                        else:
                            FP = FP + 1

                        # FALSE NEGATIVE - Pair NOT predicted but present in the solution
                        if not tagpair[1]:
                            FN = FN + 1

                        # TRUE NEGATIVE - FIXME: We won't have such cases in the current dataset format \
                        # We need to improve this
            
        except KeyError:
            pass # Ignore it for now

    precision = 0
    recall = 0
    F1 = 0
    if TP == 0:
        Precision = 0
        Recall = 0
    else: 
        precision = float(TP)/(TP + FP)        
        recall = float(TP)/(TP + FN)
        F1 = (2 * precision * recall)/(precision + recall)
    
    return {'precision': precision, 'recall': recall, 'F1': F1}

result = evaluate(result_df)
# precision = precision * 100
# print(precision, recall, F1)

In [ ]:
import pickle
import json

with open(output_file, 'wb') as handle:
    pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)

if debug:
    with open(output_file + '.txt', 'w') as f:
        f.write(json.dumps(result))
    output_df.to_csv(output_file + ".csv", encoding='ISO-8859-1', sep=",", doublequote=True, index=False)